In [10]:
######## Create, Train, and Predict Models

# Loads train, test, and val datasets
# Instantiates Models
# Trains Models
# Predicts using Models

Import All Required Modules

In [3]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
from model_binaries.utils.binary_utils import save_entity, load_entity

In [4]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations

## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_000.yaml'
yaml_path1 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_001.yaml'
yaml_path2 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_002.yaml'

yaml_paths = [yaml_path0, yaml_path1]


checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()

Load Train, Val, and Test Datasets into Memory

In [7]:
import pickle
import os

# File path to load train, test, and val datasets
train_test_val_dataset_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v01"

train_dataset_path = train_test_val_dataset_path + "/train.pkl"
val_dataset_path = train_test_val_dataset_path + "/val.pkl"
# test_dataset_path = train_test_val_dataset_path + "/test.pkl"

with open(train_dataset_path, "rb") as f:
    train_dataset = pickle.load(f)
    print("Train dataset leaded to memory")
with open(val_dataset_path, "rb") as f:
    validation_dataset = pickle.load(f)
    print("Val dataset leaded to memory")
# with open(test_dataset_path, "rb") as f:
#     test_dataset = pickle.load(f)
#     print("Test dataset leaded to memory")

Train dataset leaded to memory
Val dataset leaded to memory


Create and Train Models

In [8]:
# === STEP 4: MODEL MANAGER
print("# === STEP 4: MODEL MANAGER: Starting ...")
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)
models = model_manager.create_models([yaml_path0])

# === STEP 4: MODEL MANAGER: Starting ...


In [9]:
train_dataset.examples[0].features
len(train_dataset.examples[0].features) # should be 178

178

In [10]:
# If no val dataset, code requires to push None instead
val_dataset = validation_dataset if validation_dataset is not None else None

model_manager.train(models, [(train_dataset, val_dataset)], save_after_training=True)
print("# === STEP 4: MODEL MANAGER: Complete")

INFO:root:Training model 'e318559d1275b9459efbc58b5363fbe9' for 20 epochs with batch size 32.


Epoch 1/20
557/557 [==============================] - 6s 8ms/step - loss: 3413.9866 - mse: 3413.9866 - mae: 45.9491 - val_loss: 4174.7715 - val_mse: 4174.7715 - val_mae: 63.1986
Epoch 2/20
557/557 [==============================] - 2s 4ms/step - loss: 412.6510 - mse: 412.6510 - mae: 16.0211 - val_loss: 1012.0045 - val_mse: 1012.0045 - val_mae: 28.9775
Epoch 3/20
557/557 [==============================] - 2s 4ms/step - loss: 264.3955 - mse: 264.3955 - mae: 12.9088 - val_loss: 218.5742 - val_mse: 218.5742 - val_mae: 11.6637
Epoch 4/20
557/557 [==============================] - 3s 5ms/step - loss: 232.8006 - mse: 232.8006 - mae: 12.1361 - val_loss: 206.2176 - val_mse: 206.2176 - val_mae: 11.3258
Epoch 5/20
557/557 [==============================] - 2s 4ms/step - loss: 226.1146 - mse: 226.1146 - mae: 11.9325 - val_loss: 205.3823 - val_mse: 205.3823 - val_mae: 11.3010
Epoch 6/20
557/557 [==============================] - 2s 4ms/step - loss: 215.6276 - mse: 215.6276 - mae: 11.6565 - val_loss

INFO:root:Model 'e318559d1275b9459efbc58b5363fbe9': Finished training.


Model saved successfully at: models/e318559d1275b9459efbc58b5363fbe9/model_weights_e318559d1275b9459efbc58b5363fbe9.pth
# === STEP 4: MODEL MANAGER: Complete


Accuracy When Predicting Game Winner

In [11]:
predictions = []
for model in models:
    print(model)
    predictions.append(model.predict(validation_dataset.examples, return_target_labels=True))

In [12]:
total_examples = 0
correct_predictions = 0
correct_b_predictions = 0

for row in predictions[0].itertuples(index=False):
    total_examples += 1
    
    # Compare predictions and target labels for the current feature
    if (row.final_score_A > row.final_score_B and row.target_final_score_A > row.target_final_score_B):
        correct_predictions += 1
    elif (row.final_score_A < row.final_score_B and row.target_final_score_A < row.target_final_score_B):
        correct_predictions += 1
        correct_b_predictions += 1  

# Calculate accuracy
accuracy = correct_predictions / total_examples
percent_of_b_in_total_correct_pred = correct_b_predictions / correct_predictions
print(f"Accuracy across all features on validation dataset: {accuracy} and percent_of_b_in_total_correct_pred: {percent_of_b_in_total_correct_pred}")

Accuracy across all features on validation dataset: 0.6046633481791983 and percent_of_b_in_total_correct_pred: 0.0


In [10]:
%tensorboard --logdir logs/fit/39cf74dd0ecb478f456add01f6470ccc

UsageError: Line magic function `%tensorboard` not found.
